In [2]:
import warnings
import logging
import torch
from datasets import load_dataset
from transformers import AutoModelForVision2Seq, AutoProcessor, LlavaForConditionalGeneration
warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)
from qwen_vl_utils import process_vision_info

import os
from dataclasses import dataclass, field
from typing import Optional

2025-07-09 14:09:43,360 - INFO - set VIDEO_TOTAL_PIXELS: 90316800


In [3]:
import trl
print("trl version:", trl.__version__)

trl version: 0.19.1


In [4]:
from dataclasses import dataclass, field, asdict

In [5]:
parser = TrlParser((ScriptArguments, SFTConfig, ModelConfig))
script_args, training_args, model_args = parser.parse_args_and_config()

# for Multi-gpu DDP training with SFT Trainer
training_args.gradient_checkpointing_kwargs = dict(use_reentrant=False)
training_args.remove_unused_columns = False
training_args.dataset_kwargs = {"skip_prepare_dataset": True}

log_config = {**asdict(script_args), **asdict(training_args), **asdict(model_args)}
logging.info(f"Training config: {log_config}")

usage: ipykernel_launcher.py [-h] [--dataset_name DATASET_NAME]
                             [--dataset_config DATASET_CONFIG]
                             [--dataset_train_split DATASET_TRAIN_SPLIT]
                             [--dataset_test_split DATASET_TEST_SPLIT]
                             [--dataset_streaming [DATASET_STREAMING]]
                             [--gradient_checkpointing_use_reentrant [GRADIENT_CHECKPOINTING_USE_REENTRANT]]
                             [--ignore_bias_buffers [IGNORE_BIAS_BUFFERS]]
                             [--output_dir OUTPUT_DIR]
                             [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                             [--do_train [DO_TRAIN]] [--do_eval [DO_EVAL]]
                             [--do_predict [DO_PREDICT]]
                             [--eval_strategy {no,steps,epoch}]
                             [--prediction_loss_only [PREDICTION_LOSS_ONLY]]
                             [--per_device_train_batch_size PER_DEVIC

SystemExit: 2

/home/ubuntu/ntu/brandon/prm-training-code/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
images = ["obama.png"]
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Who is this?"},
            {"type": "image"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Barack Obama"}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is he famous for?"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "He is the 44th President of the United States."}
        ]
    }
]

from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
print(processor.apply_chat_template(messages, tokenize=False))

/home/ubuntu/ntu/brandon/prm-training-code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


USER: <image>
Who is this? ASSISTANT: Barack Obama USER: What is he famous for? ASSISTANT: He is the 44th President of the United States. 


In [2]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
print(processor.apply_chat_template(messages, tokenize=False))

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Who is this?<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
Barack Obama<|im_end|>
<|im_start|>user
What is he famous for?<|im_end|>
<|im_start|>assistant
He is the 44th President of the United States.<|im_end|>

